In [ ]:
with open('/Users/veprincevadasa/article_godunov.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
#Токенизация с NLTK

import nltk

nltk_tokens = nltk.word_tokenize(text, language='russian')

with open('/Users/veprincevadasa/tokens_nltk.txt', 'w', encoding='utf-8') as output_file:
   for token in nltk_tokens:
       output_file.write(token + '\n')

In [ ]:
#Токенизация с Stanza

import stanza
stanza.download('ru')

stanza_nlp = stanza.Pipeline('ru')
stanza_doc = nlp(text)
stanza_tokens = [word.text for sentence in stanza_doc.sentences for word in sentence.words]

with open('/Users/veprincevadasa/tokens_stanza.txt', 'w', encoding='utf-8') as output_file:
   for token in stanza_tokens:
       output_file.write(token + '\n')

In [23]:
#from razdel import tokenize
razdel_tokens = list(tokenize(text))
razdel_doc = [token.text for token in razdel_tokens]
with open('/Users/veprincevadasa/tokens_razdel.txt', 'w', encoding='utf-8') as output_file:
   for token in razdel_doc:
       output_file.write(token + '\n')

In [ ]:
#Токенизация с segtok НЕ РАБОТАЕТ

# !pip install segtok

# from segtok import tokenizer

# with open('/Users/veprincevadasa/article_godunov.txt', 'r', encoding='utf-8') as file:
#     text = file.read()
    
# segtok_tokens = tokenizer.tokenize(text)

# with open('/Users/veprincevadasa/tokens_segtok.txt', 'w', encoding='utf-8') as output_file:
#    for token in segtok_tokens:
#        output_file.write(token + '\n')

In [ ]:
# Токенизация с spacy НЕ РАБОТАЕТ
# import spacy
# spacy_nlp = spacy.load('/Users/veprincevadasa/myenv/lib/python3.12/site-packages/ru_core_news_sm')

# with open('/Users/veprincevadasa/article_godunov.txt', 'r', encoding='utf-8') as file:
#     text = file.read()
    
# spacy_doc = spacy_nlp(text)
# spacy_tokens = [token.text for token in spacy_doc]
# print("SpaCy:", spacy_tokens)

In [ ]:
#!pip install sacremoses
from sacremoses import MosesTokenizer
tokenizer = MosesTokenizer(lang='ru')
tokens_moses = tokenizer.tokenize(text)

print("Tokens (Moses):", tokens_moses)
with open('/Users/veprincevadasa/tokens_moses.txt', 'w', encoding='utf-8') as output_file:
   for token in tokens_moses:
       output_file.write(token + '\n')

In [ ]:
#!pip install pymorphy3
import pymorphy3
morph = pymorphy3.MorphAnalyzer()
tokens_pymorphy = [morph.parse(word)[0].normal_form for word in text.split()]
with open('/Users/veprincevadasa/tokens_pymorphy.txt', 'w', encoding='utf-8') as output_file:
   for token in tokens_pymorphy:
       output_file.write(token + '\n')

In [100]:
# Токенизация с udpipe
!pip install ufal.udpipe
import ufal.udpipe
model_path = '/Users/veprincevadasa/Downloads/russian-syntagrus-ud-2.5-191206.udpipe'
model = ufal.udpipe.Model.load(model_path)

pipeline = ufal.udpipe.Pipeline(model, 'tokenize', 'true', 'false', 'conllu')
udpipe_output = pipeline.process(text)

tokens_udpipe = []
for line in udpipe_output.split('\n'):
    if line and not line.startswith('#'):  # Игнорируем пустые строки и комментарии
        parts = line.split('\t')
        tokens_udpipe.append(parts[1])  # Добавляем только токен (второй элемент)


with open('/Users/veprincevadasa/tokens_udpipe.txt', 'w', encoding='utf-8') as output_file:
    for token in tokens_udpipe:
        output_file.write(token + '\n')

In [ ]:
def read_tokens(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip().splitlines()

file_paths = [
    '/Users/veprincevadasa/tokenization/tokens_nltk.txt',
    '/Users/veprincevadasa/tokenization/tokens_stanza.txt',
    '/Users/veprincevadasa/tokenization/tokens_razdel.txt',
    '/Users/veprincevadasa/tokenization/tokens_moses.txt',
    '/Users/veprincevadasa/tokenization/tokens_udpipe.txt',
    '/Users/veprincevadasa/tokenization/tokens_pymorphy.txt'
]


def compare_multiple_files(file_paths):
    all_tokens = {}
    unique_tokens = {file_path: [] for file_path in file_paths}

    for file_path in file_paths:
        all_tokens[file_path] = read_tokens(file_path)

    for file_path, tokens in all_tokens.items():
        unique = [token for token in tokens if not any(token in other_tokens for other_tokens in all_tokens.values() if other_tokens != tokens)]
        unique_tokens[file_path] = unique

    return unique_tokens

unique_tokens = compare_multiple_files(file_paths)

for file_path, tokens in unique_tokens.items():
    print(f"Токены только в {file_path}: {tokens}")

In [ ]:
# Выводы

# nltk: простой и удобный способ; ошибок нет;
# tokens_moses: делит некоторые слова (напр., привле/кательность);
# udpipe: не выделяет знак '«' как отдельный токен, привязвает его к слову (напр., «Петербургского/листка»;
# stanza: ошибок нет;
# pymorphy: не нашла код для проведения только токенизации; вместе с токенизацией провел лемматизацию, но не для всех слов (почему-то…); не выделяет знак « как отдельный токен; самый неудачный результат токенизации;
# razdel: простой и удобный способ; ошибок нет.